In [1]:
import os
import sys
import rasterio
import tensorflow as tf
from tensorflow import keras

import WINMOL_Analyzer as Analyzer
from WINMOL_Analyzer import Timer

from Config import Config

# Root directory of the project
ROOT_DIR = os.path.abspath(".//")

sys.path.append(ROOT_DIR)

# Directory whre the trained models are stored
model_dir = os.path.join(ROOT_DIR, "model//")

# Directory where the inpupt UAV-orthomosaics are stored
image_dir = os.path.join(ROOT_DIR, "input//")

# Directory to save the semantic stem map
pred_dir = os.path.join(ROOT_DIR, "pred//")

# Directory to save logs and trained model
output_dir = os.path.join(ROOT_DIR, "output//")

In [2]:
config = Config()
config.display()


Configurations:
IMG_bit                        8
IMG_height                     512
IMG_width                      512
max_distance                   8
max_tree_height                32
min_length                     2.0
n_Channels                     3
num_classes                    1
overlapp_pred                  8
tile_size                      15
tolerance_angle                7




In [3]:


# Load your pre-trained U-Net model
model_path=os.path.join(model_dir, "UNet_SpecDS_UNet_Mask-RCNN_512_Spruce_2_model_2023-02-27_061925.hdf5")

# Load the model from the HDF5 file
model = keras.models.load_model(model_path, compile=False)

# Summary of the loaded model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 64  1728        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 512, 512, 64  256        ['conv2d_1[0][0]']               
 rmalization)                   )                                                             

In [6]:
img_list=os.listdir(image_dir)
img_list= [i for i in img_list if i.endswith((".tiff",".tif")) ]
img_name=img_list[1]
print(img_name)
file_name = os.path.splitext(os.path.basename(img_name))[0]

20220212_Barnekow_4.tiff


In [7]:
#Predict with resampling per tile

tt = Timer()
tt.start()

img, profile =Analyzer.IO.load_orthomosaic(image_dir+img_name, config)

pred, profile=Analyzer.Pred.predict_with_resampling_per_tile(img, profile, model,config)   
Analyzer.IO.export_stem_map(pred, profile, pred_dir, file_name)       

segments = Analyzer.Skel.find_segments(pred, config, profile)
segments = Analyzer.Vec.restore_geoinformation(segments, config, profile)
stems = Analyzer.Vec.build_stem_parts(segments)
stems = Analyzer.Vec.connect_stems(stems,config)  
endnodes = Analyzer.Vec.rebuild_endnodes_from_stems(stems)   
stems = Analyzer.Quant.quantify_stems(stems, pred, profile,pred_dir+'pred_'+file_name+'.tiff')

Analyzer.IO.stems_to_geojson(stems, output_dir+file_name)

tt.stop()    


#######################################################
Prediction of the semantic stem map
Resampling tiles while analyzing
182  tiles analyzed
Elapsed time: 10.5453 seconds
#######################################################

#######################################################
Skeletonize Image
Elapsed time: 2.3889 seconds
#######################################################

#######################################################
Splitting the skeleton into segments and detecting endnodes
Dense nodes removed:  207
Brachpoints removed:  4582
Detected endnodes:  6988
Elapsed time: 30.4486 seconds
#######################################################

#######################################################
Find connected segments in the skeleton
Initial length of skeleton:  156248
Number of endnodes 6988
Minimum length in pixel:  17
Detected skeleton segments:  1450
Elapsed time: 144.8957 seconds
#######################################################

####################